In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
from tools import tools
from tools import Preprocessing as pp

import tensorflow as tf

%matplotlib inline

In [2]:
import os
# GPU Setting
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [3]:
print("Preparing Dataset...")
testset=pp.config(pp.csv_file)
img_set,coor_set=pp.prepare_dataset.prepare(testset)

Preparing Dataset...


/home/hahnz/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
Classes=np.array(["Right ankle", "Right knee", "Right hip", "Left hip", "Left knee", "Left ankle", "Right wrist",
                  "Right elbow", "Right shoulder", "Left shoulder", "Left elbow", "Left wrist", "Neck", "Head top"])

In [5]:
X = tf.placeholder(tf.float32, [None, 256,256,3])
y = tf.placeholder(tf.float32, [None, 14])

In [6]:
def _conv(data,output,ksize,padding,name):
    return tf.layers.conv2d(data, filters=output, kernel_size=ksize, padding=padding, activation=tf.nn.relu, name=name)

def _bn(data):
    return tf.contrib.layers.batch_norm(data)

def _max_pooling(data,name):
    return tf.nn.max_pool(data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name=name)

In [7]:
def _conv_block(idx,tensor,filters,ksize,padding):
    conv1 = _conv(tensor, filters, ksize, padding, "conv"+str(idx)+"_1")
    bn1 = _bn(conv1)

    conv2 = _conv(bn1, filters, ksize, padding, "conv"+str(idx)+"_2")
    bn2 = _bn(conv2)

    pool = _max_pooling(bn2, "pool"+str(idx))
    
    return pool
    def get_shape():
        model_shape = list

In [8]:
def fc(x, num_in, num_out, name, relu=True):
    
    with tf.variable_scope(name) as scope:
        weights = tf.get_variable('weights', shape=[num_in, num_out], trainable=True)
        biases = tf.get_variable('biases', [num_out], trainable=True)
        act = tf.nn.xw_plus_b(x, weights, biases, name=scope.name)

    if relu : return tf.nn.relu(act)
    else: return act

In [9]:
def dropout(x):
    return tf.nn.dropout(x, 0.5)

In [10]:
conv_layer1 = _conv_block(1,X,32,11,"SAME")

conv_layer2 = _conv_block(2,conv_layer1,64,7,"SAME")

conv_layer3 = _conv_block(3,conv_layer2,128,5,"SAME")

conv_layer4 = _conv_block(4,conv_layer3,256,3,"SAME")

num_nodes=int(conv_layer4.get_shape()[1])*int(conv_layer4.get_shape()[2])*int(conv_layer4.get_shape()[3])

reshaped_conv4 = tf.reshape(conv_layer4, [-1, num_nodes])

fc5 = fc(reshaped_conv4, num_nodes, 4096, name="fc5")
drop5 = dropout(fc5)

fc6 = fc(drop5, 4096, Classes.shape[0], name="fc6")

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
# training Codes